In [1]:
from sys import stdout
import numpy as np
from mdtraj.reporters import HDF5Reporter
import openmm as mm
from openmm import app
from openmm import unit
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
from openff.toolkit.topology import Molecule
from pdbfixer import PDBFixer

import nglview as nv
import mdtraj as md

## Load protein file and fix it
Load the unfixed protein file without water

In [2]:
protein_file = "../1_system_inspection/receptor_1.pdb"
fixer = PDBFixer(filename=protein_file)

fixer.findMissingResidues()
missing_residues = fixer.missingResidues
fixer.findNonstandardResidues()
nonstandard_residues = fixer.nonstandardResidues
fixer.findMissingAtoms()
missing_atoms = fixer.missingAtoms
missing_terminals = fixer.missingTerminals
fixer.addMissingAtoms()

In [3]:
app.PDBFile.writeFile(fixer.topology, fixer.positions, open('receptor_unsolvated.pdb', 'w'))

## Load protein topology and positions

In [4]:
modeller = app.Modeller(fixer.topology, fixer.positions)

## Load ligand topology and positions

In [5]:
ligand_file = "../3_ligand_preparation/ligand.sdf"
ligand = Molecule.from_file(ligand_file)
ligand_positions = ligand.conformers[0]
ligand_topology = ligand.to_topology()
modeller.add(ligand_topology.to_openmm(), ligand_positions)

## Load forcefield

In [6]:
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-1.3.0.offxml", molecules=[ligand])
forcefield.registerTemplateGenerator(smirnoff.generator)

In [7]:
residues_protonated = modeller.addHydrogens(forcefield=forcefield, pH=7.2)

/bin/bash: /home/daniel/miniconda3/envs/cheminformatics/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/daniel/miniconda3/envs/cheminformatics/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/daniel/miniconda3/envs/cheminformatics/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/daniel/miniconda3/envs/cheminformatics/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/daniel/miniconda3/envs/cheminformatics/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/daniel/miniconda3/envs/cheminformatics/lib/libtinfo.so.6: no version information available (required by /bin/bash)


## Add solvent

In [8]:
clearance = 14*unit.angstroms
max_size = max(max((pos[i] for pos in modeller.positions))-min((pos[i] for pos in modeller.positions)) for i in range(3))
vectors = mm.Vec3(1.0, 0, 0), mm.Vec3(1.0/3.0, 2.0*np.sqrt(2.0)/3.0,0.0), mm.Vec3(-1.0/3.0, np.sqrt(2.0)/3.0, np.sqrt(6.0)/3.0)
box_vectors = [(max_size + clearance)*v for v in vectors]

In [9]:
modeller.addSolvent(forcefield, model='tip3p', boxVectors = box_vectors, neutralize=True)

In [10]:
# Verify that water and ions were added
n_waters = 0
ions = []
for chain in modeller.topology.chains():
    for residue in chain.residues():
        if residue.name=='HOH':
            n_waters += 1
        if len(list(residue.atoms()))==1:
            ions += [atom.name for atom in residue.atoms()]            

print('n_waters: {}'.format(n_waters))
print('ions: {}'.format(ions))

n_waters: 7773
ions: ['Na', 'Na', 'Na', 'Na']


## System creation

In [11]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer,
                                 constraints=app.HBonds, rigidWater=True)

for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        nonbondedForce = force

nonbondedForce.setUseDispersionCorrection(True)
nonbondedForce.setEwaldErrorTolerance(1.0e-5)
nonbondedForce.setUseSwitchingFunction(True)
nonbondedForce.setSwitchingDistance(0.8*unit.nanometer)

## Platform

In [12]:
platform = mm.Platform.getPlatformByName('CUDA')

## Integrator

In [14]:
integration_timestep = 2.0*unit.femtoseconds
temperature = 300.0*unit.kelvin

integrator = mm.LangevinIntegrator(temperature, 1.0/unit.picosecond, integration_timestep)

## Simulation object

In [15]:
simulation = app.Simulation(modeller.topology, system, integrator, platform)

## Initial coordinates

In [16]:
simulation.context.setPositions(modeller.positions)

In [17]:
# Check the potential energy before minimization
initial_state = simulation.context.getState(getEnergy=True)
initial_state.getPotentialEnergy()

Quantity(value=42622666032047.2, unit=kilojoule/mole)

In [18]:
simulation.minimizeEnergy()

In [ ]:
# Check the potential energy after minimization
minimized_state = simulation.context.getState(getEnergy=True)
minimized_state.getPotentialEnergy()

In [20]:
# Writing out the minimized system
minimized_positions = simulation.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(simulation.topology, minimized_positions, open('initial.pdb', 'w'))

In [21]:
view = nv.show_structure_file("initial.pdb")
view.add_licorice(selection="(not protein)")
view

NGLWidget()

## Initial velocities

In [22]:
simulation.context.setVelocitiesToTemperature(temperature)

## Run a short simulation

In [23]:
simulation_time = 10000.0*unit.picoseconds
reporting_time = 20.0*unit.picoseconds
saving_time = 10.0*unit.picoseconds

simulation_steps = int(simulation_time/integration_timestep)
reporting_steps = int(reporting_time/integration_timestep)
saving_steps = int(saving_time/integration_timestep)

print(f"{simulation_steps} steps")
print(f"{reporting_steps} reporting steps")
print(f"{saving_steps} saving steps")

5000000 steps
10000 reporting steps
5000 saving steps


In [24]:
states_reporter = app.StateDataReporter(stdout, reporting_steps, step=True, potentialEnergy=True, temperature=True,
                                    progress=True, totalSteps=simulation_steps, speed=True, remainingTime=True)
traj_reporter = HDF5Reporter('traj.h5', saving_steps, coordinates=True, time=True, cell=True, potentialEnergy=True,
                             kineticEnergy=True, temperature=True)
simulation.reporters+=[states_reporter, traj_reporter]

In [25]:
simulation.step(simulation_steps)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Time Remaining"
0.2%,10000,-332001.5458761598,300.97685634124076,0,--
0.4%,20000,-333009.0458761598,300.1459181877383,24.7,9:40:42
0.6%,30000,-332336.4208761598,300.9644323712823,24.8,9:38:04
0.8%,40000,-332230.6708761598,303.8533042388448,24.8,9:36:24
1.0%,50000,-333282.5458761598,300.6339666662846,24.8,9:34:57
1.2%,60000,-332951.5458761598,297.06260690940616,24.8,9:34:17
1.4%,70000,-333790.4208761598,304.4809593196949,24.7,9:33:48
1.6%,80000,-333529.2958761598,299.18668756832227,24.7,9:33:50
1.8%,90000,-332851.0458761598,301.61901339296463,24.7,9:32:46
2.0%,100000,-333612.7958761598,299.86301769863087,24.7,9:31:23
2.2%,110000,-330873.4208761598,298.1697514553537,24.7,9:30:05
2.4%,120000,-333607.0458761598,300.0780847456501,24.7,9:29:40
2.6%,130000,-332716.0458761598,301.23922730202486,24.7,9:28:41
2.8%,140000,-333216.0458761598,303.57396480567536,24.7,9:27:38
3.0%,150000,-331970.0458761598,302.920

In [26]:
traj_reporter.close()

In [27]:
# Writing out the final system
final_positions = simulation.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(simulation.topology, final_positions, open('final.pdb', 'w'))

# Visualize Simulation

In [29]:
topology = modeller.getTopology()
traj = md.load("traj.h5", top=md.Topology.from_openmm(topology))

view = nv.show_mdtraj(traj)
# view.add_licorice(selection="(not protein)") # View water molecules
view

NGLWidget(max_frame=999)